In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
import random as rn
from sklearn.metrics.pairwise import cosine_similarity
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)


In [ ]:
path =   # Enter the directory of your project folder
PATH = path + "/Results"

if not os.path.exists(PATH):
    os.mkdir(PATH) 

In [ ]:
df = pd.read_csv(path+"//ToyDataset.csv").drop(['Unnamed: 0'],axis=1)
df1 = pd.read_csv(PATH+"//Data one hot encoded.csv").drop(['Unnamed: 0'],axis=1)
clientID = df.ClientID.unique()

df["EntryDate"] = pd.to_datetime(df['EntryDate'])
df["ExitDate"] = pd.to_datetime(df['ExitDate'])
df = df.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)
df1["EntryDate"] = pd.to_datetime(df1['EntryDate'])
df1["ExitDate"] = pd.to_datetime(df1['ExitDate'])
df1 = df1.sort_values(["EntryDate",'ExitDate']).reset_index(drop=True)

In [ ]:
goal = [10,11]
cycle_pd = pd.DataFrame(columns = ['ClientID','length','projectType','target'])
index = {}
for i in range(len(clientID)):
    closed = []
    temp = df[df['ClientID']==clientID[i]].reset_index(drop=True)
    for j in range(len(temp)):
        if temp['ProjectType'][j] not in closed:
            closed.append(temp['ProjectType'][j])
            temp1 = temp[temp['ProjectType']==temp['ProjectType'][j]]
            if len(temp1)>2:
                for k in range(1,len(temp1)):
                    length = temp1.index[k]-temp1.index[k-1]
                    if temp.Destination.iloc[-1] in goal: 
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'successful'},ignore_index=True)
                    elif np.isnan(temp.Destination.iloc[-1]):
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'abandoned'},ignore_index=True)
                    else: 
                        cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'unsuccessful'},ignore_index=True)


            elif len(temp1)==2:
                length = temp1.index[-1]-temp1.index[0]
                if temp.Destination.iloc[-1] in goal: 
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'successful'},ignore_index=True)
                elif np.isnan(temp.Destination.iloc[-1]):
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'abandoned'},ignore_index=True)                    
                else: 
                    cycle_pd = cycle_pd.append({'ClientID':clientID[i],'length':length,'projectType':temp['ProjectType'][j],'target':'unsuccessful'},ignore_index=True)



In [ ]:
cycle_goal = pd.DataFrame(columns=['ClientID',"Target","CycleNumber"])
for i in range(len(clientID)):
    temp = df[df['ClientID']==clientID[i]]
    
    if temp['Destination'].iloc[-1] in goal:
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'successful',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'successful',"cycle":'no',"CycleNumber":"None"},ignore_index=True)
    
    elif np.isnan(temp['Destination'].iloc[-1]):
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'abandoned',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'abandoned',"cycle":'no',"CycleNumber":"None"},ignore_index=True)

    else:
        if clientID[i] in np.array(cycle_pd['ClientID']):
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'unsuccessful',"cycle":'yes',"CycleNumber":len(cycle_pd[cycle_pd["ClientID"]==clientID[i]])},ignore_index=True)
        else:
            cycle_goal = cycle_goal.append({"ClientID":clientID[i],"Target":'unsuccessful',"cycle":'no',"CycleNumber":"None"},ignore_index=True)



In [ ]:
target_unique = cycle_goal.Target.unique()
for t in target_unique:
    temp = cycle_goal[cycle_goal['Target']==t]
    average = temp[temp['CycleNumber']!='None']['CycleNumber'].mean()
    print(t,average)
    

In [ ]:
target_unique = cycle_pd.target.unique()
for t in target_unique:
    temp = cycle_pd[cycle_pd['target']==t]
    average = temp['length'].mean()
    print(t,average)
    

In [ ]:
cycle_length = pd.DataFrame(columns = ['ClientID','mean_length','target'])
clientID = cycle_pd.ClientID.unique()

for i in range(len(clientID)):
    temp = cycle_pd[cycle_pd['ClientID']==clientID[i]]
    target = temp['target'].iloc[-1]
    mean_length = temp['length'].mean()
    cycle_length = cycle_length.append({'ClientID':clientID[i],'mean_length':mean_length,'target':target},ignore_index=True)
   


In [ ]:
target_unique = cycle_length.target.unique()
for t in target_unique:
    temp = cycle_length[cycle_length['target']==t]
    average = temp['mean_length'].mean()
    print(t,average)
    